In [1]:
!nvidia-smi 

/bin/bash: nvidia-smi: command not found


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Code' 
%run Unet_2D.ipynb
%run preprocessing_2D.ipynb
%run otherUnets.ipynb

torch.Size([1, 2, 192, 256])
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
66882690


In [3]:
!pip install pytorch-lightning
!pip install timm
import pytorch_lightning as pl
from IPython.display import clear_output
import nibabel as nib
import csv
import os
import glob 
import torch
import timm.optim
import torch.nn as nn
from timm.data.loader import create_loader
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
from fastprogress import master_bar, progress_bar
from torchvision import transforms
import torch.optim as optim
from itertools import product
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class HistoryLogger(pl.callbacks.Callback):
    def __init__(self, dir = "history_isic_AC.csv"):
        self.dir = dir
    def on_validation_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        if "loss_epoch" in metrics.keys():
            logs = {"epoch": trainer.current_epoch}
            keys = ["loss_epoch", "train_dice_epoch", "train_jac_epoch", "val_loss","val_dice", "val_jac"]
            for key in keys:
                logs[key] = metrics[key].item()
            header = list(logs.keys())
            isFile = os.path.isfile(self.dir)
            with open(self.dir, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=header)
                if not isFile:
                    writer.writeheader()
                writer.writerow(logs) 
        else:
            pass
def setDropProb(model, prob=0.01):
    for layer in model.modules():
        if isinstance(layer, DropBlock2D):
            layer.drop_prob = prob
clear_output()
############## turn off Debug APIs for Final Training############
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

In [ ]:
#ISIC18
%cd '/content/drive/MyDrive/Python/Biomedical_Image_Segmentation' 
data = np.load("/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/ISIC2018_train.npz")
x, y = data["image"], data["mask"]
test_size = int((20/100)*x.shape[0])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=520, random_state=312)
del x, y, data

/content/drive/MyDrive/Python/Biomedical_Image_Segmentation


In [4]:
#ISIC17
data = np.load("/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/ISIC2017_192x256_test.npz")
x, y = data["image"], data["mask"]
test_size = int((20/100)*x.shape[0])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=312)
del x, y, data

In [ ]:
#PH2
data = np.load('/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/PH2_192_256.npz')
X_train, Y_train = data["image"], data["mask"]
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=30, random_state=312)

In [5]:
columns = ['0','1']
df = pd.DataFrame(columns=columns)
_, counts =  np.unique(y_train, return_counts=True)
conts_dict = dict(zip(columns, counts))
df = df.append(conts_dict, ignore_index=True)
label_0 = df['0'].sum()
label_1 = df['1'].sum()
total_labels = label_0 + label_1
n_classes = 2
#Class weights claculation: n_samples / (n_classes * n_samples_for_class)
wt0 = round((total_labels/(n_classes*label_0)), 2)
wt1 = round((total_labels/(n_classes*label_1)), 2)
print("\n Weights are:", wt0, wt1)


 Weights are: 0.62 2.63


<ipython-input-5-1977a65b1be0>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(conts_dict, ignore_index=True)


In [6]:
train_dataset = DataLoader(ISICLoader(x_train, y_train), batch_size=8, pin_memory=True, shuffle=True, num_workers=2, drop_last=True, prefetch_factor = 8)
test_dataset = DataLoader(ISICLoader(x_test, y_test, typeData="test"), batch_size=16, num_workers=2, prefetch_factor=16)

In [7]:
class SemiActiveLoss(nn.Module):
    def __init__(self, device, alpha =1e-9, beta = 1e-1, lamda = 1e-3):
        super().__init__()
        self.device = device
        self.alpha = alpha
        self.beta = beta      
        self.lamda = lamda
    def LevelsetLoss(self, image, y_pred, kernel_size=5, smooth=1e-5):
        kernel = torch.ones(1, y_pred.size(1), kernel_size, kernel_size, device=self.device) / kernel_size**2
        padding = kernel_size //2
        lossRegion = 0.0
        y_pred_fuzzy = y_pred ** 2
        for ich in range(image.size(1)):
            target_ = image[:,ich:ich+1] 
            pcentroid_local = F.conv2d(target_ * y_pred_fuzzy + smooth, kernel, padding = padding) \
                                / F.conv2d(y_pred_fuzzy + smooth, kernel, padding = padding)
            plevel_local = target_ - pcentroid_local
            loss_local = plevel_local * plevel_local * y_pred_fuzzy

            pcentroid_global = torch.sum(target_ * y_pred_fuzzy, dim=(2,3),keepdim=True) \
                                / torch.sum(y_pred_fuzzy+smooth, dim=(2,3),keepdim = True)   
            plevel_global = target_ - pcentroid_global
            loss_global = plevel_global * plevel_global * y_pred_fuzzy

            lossRegion += torch.sum(loss_local) + self.beta * torch.sum(loss_global)
        return lossRegion 
    def GradientLoss(self, y_pred, penalty = "l1"):
        dH = torch.abs(y_pred[...,1:] - y_pred[...,:-1])
        dW = torch.abs(y_pred[:,:,1:] - y_pred[:,:,:-1])
        if penalty == "l2":
            dH = dH * dH
            dW = dW * dW
        loss =  torch.sum(dH) +  torch.sum(dW)
        return loss
    def ActiveContourLoss(self, y_true, y_pred, smooth=1e-6):   
        dim = (1,2,3)
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:,1:]
        y_pred = y_pred[:,1:]

        active = - torch.log(1-y_pred+smooth) * (1-yTrueOnehot) - torch.log(y_pred+smooth) * yTrueOnehot
        loss = torch.sum(active, dim = dim) / torch.sum(yTrueOnehot + y_pred - yTrueOnehot * y_pred + smooth, dim = dim)
        return torch.mean(loss)

    def forward(self, image, y_true, y_pred):
        active = self.ActiveContourLoss(y_true, y_pred)
        # levelset =  self.LevelsetLoss(image, y_pred)
        # length = self.GradientLoss(y_pred)
        return active
        # return active + self.alpha * (levelset + self.lamda * length) 

In [8]:
class ActiveFocalLoss(nn.Module):
    def __init__(self, device, class_weight=[1] * NUM_CLASS, gamma=2):
        """
        class weight should be a list. 
        """
        super().__init__()
        self.device = device
        self.class_weight = torch.tensor(class_weight, device=device)
        self.gamma = gamma
    def forward(self, y_true, y_pred):
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)
        y_pred = torch.clamp(y_pred,min=1e-7,max=1-1e-7)

        active_focal = - yTrueOnehot * (1-y_pred)**self.gamma * torch.log(y_pred) \
                        - (1 - yTrueOnehot) * y_pred**self.gamma * torch.log(1 - y_pred)
        loss = torch.sum(active_focal, dim=[2, 3]) * self.class_weight
        return torch.sum(loss) / (torch.sum(self.class_weight) * y_true.size(0) * y_true.size(2) * y_true.size(3))

class ActiveContourLoss(nn.Module):
    def __init__(self, device, class_weight=[1] * NUM_CLASS):
        """
        class weight should be a list. 
        """
        super().__init__()
        self.device = device
        self.class_weight = torch.tensor(class_weight, device=device)
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)

        # y_pred = torch.clamp(y_pred,min=1e-7,max=1-1e-7)
        active = torch.sum(yTrueOnehot * (1 - y_pred) + (1 - yTrueOnehot) * y_pred, dim=[2, 3])
        loss = torch.sum(active * self.class_weight)
        return loss / (torch.sum(self.class_weight) * y_true.size(0) * y_true.size(2) * y_true.size(3))

In [9]:
class DiceLoss(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.device = device
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:, 1:]
        y_pred = y_pred[:,1:]

        intersection = torch.sum(yTrueOnehot * y_pred, dim=[1,2,3])
        cardinality  = torch.sum(yTrueOnehot + y_pred , dim=[1,2,3])
        loss = 1.0-torch.mean((2. * intersection + 1e-5) / (cardinality + 1e-5))
        return loss

In [10]:
class TverskyLoss(nn.Module):
    def __init__(self, device, alpha = 0.7):
        super().__init__()
        self.device = device
        self.alpha = alpha
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:, 1:]
        y_pred = y_pred[:,1:]

        TP = torch.sum(yTrueOnehot * y_pred, dim=[1,2,3])
        FN = torch.sum(yTrueOnehot * (1-y_pred), dim=[1,2,3])
        FP = torch.sum((1-yTrueOnehot) * y_pred, dim=[1,2,3])
        loss = 1-torch.mean((TP + 1e-5) / (TP + self.alpha*FN + (1-self.alpha)*FP + 1e-5))
        return loss

In [11]:
%run Unet_2D.ipynb

In [12]:
class Segmentor(pl.LightningModule):
    def __init__(self, model = SegNet()):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items

    # def _step(self, batch):
    #     image, y_true = batch
    #     y_pred = self.model(image)
    #     loss = SemiActiveLoss(device=self.device)(image, y_true, y_pred)
    #     dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
    #     return loss, dice_score, jaccard_score    
    def _step(self, batch):
        image, y_true = batch
        y_pred = self.model(image)
        loss_focal = ActiveFocalLoss(self.device, [0.64, 2.34])(y_true, y_pred)
        loss_contour = ActiveContourLoss(self.device, [0.64, 2.34])(y_true, y_pred)
        loss = loss_focal + loss_contour
        dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
        return loss, dice_score, jaccard_score

    def training_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"loss": loss, "train_dice": dice_score, "train_jac": jaccard_score}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"val_loss": loss, "val_dice": dice_score, "val_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"test_loss": loss, "test_dice": dice_score, "test_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics
        

    def configure_optimizers(self):
        optimizer = timm.optim.Nadam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor = 0.5, patience=16, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "val_dice"}
        return [optimizer], lr_schedulers
    

In [13]:
segmentor = Segmentor(SegNet()) #Segmentor(U_Net(drop_prob=0)) 
os.makedirs('/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/SegNet', exist_ok = True) 
check_point = pl.callbacks.model_checkpoint.ModelCheckpoint("/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/SegNet", filename="ckpt{val_dice:0.4f}",
                                                            monitor="val_dice", mode = "max", save_top_k =1,
                                                            verbose=True, save_weights_only=True,
                                                            auto_insert_metric_name=False,)
progress_bar = pl.callbacks.TQDMProgressBar(refresh_rate=1)
logger = HistoryLogger()
swa = pl.callbacks.StochasticWeightAveraging(swa_lrs=1e-3,swa_epoch_start=25)
PARAMS = {"accelerator": "cpu", "benchmark":True, "enable_progress_bar":True, "logger":False,
        #   "callbacks" : [progress_bar],
        #    "overfit_batches" :1,
          "callbacks" : [check_point, progress_bar, logger],
          "log_every_n_steps" :1, "num_sanity_val_steps":0, "max_epochs":50,
          "precision":16,
          }

trainer = pl.Trainer(**PARAMS)



/usr/local/lib/python3.9/dist-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:517: UserWarning: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [14]:
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/SegNet/ckpt0.8796.ckpt")

In [ ]:
trainer.fit(segmentor, train_dataset, test_dataset)

In [ ]:
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Weight/AttentionUnet/ckpt0.8802.ckpt")

In [ ]:
trainer.test(segmentor, test_dataset)

Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
def predict(images, model, batch_size = 64):
    images = torch.as_tensor(images, dtype= torch.float32)
    y_preds = torch.zeros((images.size(0), NUM_CLASS, images.size(2), images.size(3)), device= device)
    batch_start = 0
    batch_end = batch_size
    pbar = tqdm()
    while batch_start < images.size(0):
        image = images[batch_start : batch_end]
        with torch.inference_mode():
            image = image.to(device)
            y_pred = model(image)
            y_preds[batch_start : batch_end] = y_pred
        batch_start += batch_size
        batch_end += batch_size
        pbar.update(1)
    pbar.close()
    res = y_preds.cpu().numpy()
    del y_preds
    return res

In [ ]:
from sklearn.metrics import recall_score, precision_score, precision_recall_fscore_support
# x_inf = np.zeros((x_test.shape[0], 3, x_test.shape[1], x_test.shape[2])) #x_inf = x_test.transpose(0, -1, 1) / 255
x_inf = x_test.transpose(0, -1, 1, 2) / 255
#segmentor = Segmentor(PiDiNet(img_channel=3, inplane=32, dil=2, sa=True, ta=False)) #Segmentor(U_Net(drop_prob=0)) 
#segmentor = Segmentor.load_from_checkpoint(checkpoint_path="")
segmentor = segmentor.to(device)
segmentor.eval()
y_pred = predict(x_inf, segmentor)
torch.cuda.empty_cache()
mask_predict = np.argmax(y_pred, axis=1)
a = y_test.reshape(y_test.shape[0], -1)
b = mask_predict.reshape(mask_predict.shape[0], -1)
print(precision_recall_fscore_support(a, b, average="samples", zero_division=1))
inter = np.sum(a * b, axis=1)
uni = np.sum(a + b,  axis=1)
dice1 = (2*inter+1e-5) / (uni+1e-5) 

uni1 = np.sum(a + b - a*b,  axis=1)
jac = (inter+1e-5) / (uni1+1e-5)
print(np.mean(dice1), np.mean(jac))


0it [00:00, ?it/s]
1it [00:02,  2.55s/it]
2it [00:05,  2.97s/it]
3it [00:09,  3.11s/it]
4it [00:12,  3.18s/it]
5it [00:15,  3.22s/it]
6it [00:18,  3.25s/it]
7it [00:20,  2.93s/it]


(0.9206481120912207, 0.8766427340863396, 0.87963558020198, None)
0.8796355804065936 0.8055675992704857


In [ ]:
%cd '/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Visualize'
np.savez_compressed("ISIC17_segnet", predicted_mask=mask_predict)

/content/drive/MyDrive/Python/Biomedical_Image_Segmentation/Visualize


In [ ]:
for i in range(x_test.shape[0]):
    plt.figure(i+1, figsize=(15,15))
    plt.subplot(131), plt.imshow(x_test[i]), plt.axis('off')
    plt.subplot(132), plt.imshow(y_test[i], cmap ='gray'), plt.axis('off')
    plt.subplot(133), plt.imshow(mask_predict[i], cmap ='gray'), plt.axis('off')
    if i == 20:
        break

Output hidden; open in https://colab.research.google.com to view.